In [5]:
import pandas as pd
import numpy as np
import skcriteria as skc
from skcriteria.preprocessing import invert_objectives, scalers
from skcriteria.agg import simple

In [3]:
%%timeit
4+4

6.04 ns ± 0.504 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [22]:
small_data = pd.read_csv('hospital_data_full.csv', engine='pyarrow').loc[:, ['Patient Survey Star Rating', 'Score', 'ID']]
large_data = pd.read_csv('synthetic_hospital_data_100k_v2.csv', engine='pyarrow').loc[:, ['Patient Survey Star Rating', 'Score', 'ID']]

In [23]:
small_data.head()

,Patient Survey Star Rating,Score,ID
0,2.090909,80.200000,1336420
1,3.181818,83.684211,136502
2,2.909091,94.483871,196784204
3,4.090909,78.642857,3236251
4,3.727273,87.227273,3336078


In [31]:
small_data

,Patient Survey Star Rating,Score,ID
0,2.090909,80.200000,1336420
1,3.181818,83.684211,136502
2,2.909091,94.483871,196784204
3,4.090909,78.642857,3236251
4,3.727273,87.227273,3336078
...,...,...,...
769,2.363636,84.181818,31733317
770,3.909091,77.083333,10071655
771,2.818182,83.033333,1771730
772,2.000000,99.692308,3392311


In [24]:
large_data.head()

,Patient Survey Star Rating,Score,ID
0,1447.797607,35605.308594,-83373449216
1,1529.498779,37330.425781,-88225284096
2,828.557068,20303.488281,-47549206528
3,1297.953125,31935.941406,-74501464064
4,1233.274170,30211.619141,-71078887424


In [25]:
weights = [0.25, 0.25, 0.5]

In [26]:
objectives = [max, max, min]

In [28]:
%%timeit
# print(state.matrix.isnull().sum().sum())
dm = skc.mkdm(
    small_data, 
    objectives, 
    weights=weights, 
    alternatives=small_data.index,
    criteria=['Patient Survey Star Rating', 'Score', 'ID'])
# print(dm)
inverter = invert_objectives.InvertMinimize()
dmt = inverter.transform(dm)
# print(dmt)
scaler = scalers.SumScaler(target="both")
dmt = scaler.transform(dmt)
# print(dmt)
dec = simple.WeightedSumModel()
rank = dec.evaluate(dmt).to_series()


2.63 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit
# print(state.matrix.isnull().sum().sum())
dm = skc.mkdm(
    large_data, 
    objectives, 
    weights=weights, 
    alternatives=large_data.index,
    criteria=['Patient Survey Star Rating', 'Score', 'ID'])
# print(dm)
inverter = invert_objectives.InvertMinimize()
dmt = inverter.transform(dm)
# print(dmt)
scaler = scalers.SumScaler(target="both")
dmt = scaler.transform(dmt)
# print(dmt)
dec = simple.WeightedSumModel()
rank = dec.evaluate(dmt).to_series()


30.8 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
